In [2]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datetime import datetime
import re
import os

class ORSocietyJobProcessor:
    def __init__(self):
        # Gendered words lists for analysis
        self.masculine_words = [
            'leader', 'competitive', 'dominant', 'assertive', 'aggressive', 'ambitious',
            'analytical', 'confident', 'decisive', 'determined', 'independent', 'objective',
            'self-reliant', 'strong', 'superior', 'lead', 'manage', 'direct', 'control',
            'drive', 'challenge', 'compete', 'win', 'achieve', 'dominate', 'excel',
            'individual', 'autonomous', 'hierarchy', 'decision', 'responsibility'
        ]
        
        self.feminine_words = [
            'collaborative', 'cooperative', 'supportive', 'nurturing', 'empathetic',
            'interpersonal', 'communicate', 'understand', 'responsible', 'connect',
            'honest', 'loyal', 'dependable', 'committed', 'dedicated', 'support',
            'help', 'assist', 'care', 'share', 'together', 'team', 'community',
            'relationship', 'trust', 'warm', 'kind', 'inclusive', 'collaborate'
        ]

    def clean_text(self, text):
        """Clean and normalize text"""
        if not text:
            return ""
        
        # Remove extra whitespace and normalize
        text = ' '.join(text.split())
        # Remove special characters but keep basic punctuation
        text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\(\)]', ' ', text)
        # Remove multiple spaces
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def extract_job_id(self, url):
        """Extract job ID from URL"""
        try:
            # Pattern for OR Society URLs: /jobs/view/title/job-id/
            match = re.search(r'/jobs/view/[^/]+/([^/]+)/?', url)
            if match:
                return match.group(1)
            return url.split('/')[-1] or url.split('/')[-2]
        except:
            return "manual_entry"

    def detect_remote_option(self, text):
        """Detect remote work options from text"""
        if not text:
            return "Not specified"
        
        text_lower = text.lower()
        remote_keywords = [
            'remote', 'work from home', 'telecommute', 'virtual', 'distributed',
            'home-based', 'location independent', 'anywhere', 'flexible location'
        ]
        hybrid_keywords = ['hybrid', 'flexible', 'part remote', 'some remote']
        
        for keyword in remote_keywords:
            if keyword in text_lower:
                return "Remote"
        
        for keyword in hybrid_keywords:
            if keyword in text_lower:
                return "Hybrid"
        
        return "On-site"

    def extract_salary_info(self, text):
        """Extract salary information from text"""
        if not text:
            return "Not specified"
        
        # Look for salary patterns
        salary_patterns = [
            r'£\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s*(?:-|to)\s*£?\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)',
            r'\$\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s*(?:-|to)\s*\$?\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)',
            r'(\d{1,3}(?:,\d{3})*)\s*(?:-|to)\s*(\d{1,3}(?:,\d{3})*)\s*(?:per year|annually|pa|p\.a\.)',
            r'£(\d{1,3}(?:,\d{3})*)', r'\$(\d{1,3}(?:,\d{3})*)', r'(\d{1,3}(?:,\d{3})*)\s*(?:per year|pa|p\.a\.)'
        ]
        
        for pattern in salary_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                if len(match.groups()) >= 2:
                    return f"{match.group(1)} - {match.group(2)}"
                else:
                    return match.group(1)
        
        return "Not specified"

    def analyze_gendered_language(self, text):
        """Analyze gendered language in text"""
        if not text:
            return 0, 0, 0, []
        
        # Clean and tokenize text
        text_lower = text.lower()
        words = re.findall(r'\b[a-z]+\b', text_lower)
        
        masculine_count = sum(1 for word in words if word in self.masculine_words)
        feminine_count = sum(1 for word in words if word in self.feminine_words)
        
        # Find gendered words present
        gendered_words_found = []
        for word in words:
            if word in self.masculine_words:
                gendered_words_found.append(f"M:{word}")
            elif word in self.feminine_words:
                gendered_words_found.append(f"F:{word}")
        
        total_words = len(words)
        if total_words == 0:
            return 0, 0, 0, gendered_words_found
        
        masculine_score = round((masculine_count / total_words) * 100, 2)
        feminine_score = round((feminine_count / total_words) * 100, 2)
        neutral_score = round(100 - masculine_score - feminine_score, 2)
        
        return masculine_score, feminine_score, neutral_score, gendered_words_found

    def determine_job_level(self, title, description):
        """Determine job level from title and description"""
        combined_text = f"{title} {description}".lower()
        
        senior_keywords = ['senior', 'lead', 'principal', 'head', 'director', 'manager', 'chief', 'vp', 'vice president']
        junior_keywords = ['junior', 'entry', 'graduate', 'intern', 'trainee', 'assistant', 'associate']
        mid_keywords = ['mid', 'intermediate', 'specialist', 'analyst', 'coordinator']
        
        for keyword in senior_keywords:
            if keyword in combined_text:
                return "Senior"
        
        for keyword in junior_keywords:
            if keyword in combined_text:
                return "Junior"
        
        for keyword in mid_keywords:
            if keyword in combined_text:
                return "Mid-level"
        
        return "Not specified"

    def get_job_input(self, url):
        """Get job information through user input"""
        print("\n" + "="*60)
        print("JOB DATA ENTRY")
        print("="*60)
        print(f"URL: {url}")
        print("-"*60)
        
        # Get basic information
        job_title = input("Job Title: ").strip()
        company_name = input("Company Name: ").strip()
        location = input("Location: ").strip()
        
        # Get optional information
        posting_date = input("Posting Date (optional): ").strip()
        salary_info = input("Salary Info (optional): ").strip()
        
        # Get job description
        print("\nJob Description:")
        print("(Paste the job description below. Press Enter twice when finished)")
        print("-" * 40)
        
        description_lines = []
        empty_line_count = 0
        
        while empty_line_count < 2:
            try:
                line = input()
                if line.strip() == "":
                    empty_line_count += 1
                else:
                    empty_line_count = 0
                description_lines.append(line)
            except EOFError:
                break
        
        description = '\n'.join(description_lines).strip()
        
        return {
            'job_title': job_title,
            'company_name': company_name,
            'location': location,
            'posting_date': posting_date,
            'salary_info': salary_info,
            'description': description
        }

    def process_job_data(self, input_data, url):
        """Process the manually entered job data"""
        
        # Clean the description
        description_cleaned = self.clean_text(input_data['description'])
        
        # Create job data structure
        job_data = {
            'job_id': self.extract_job_id(url),
            'platform': 'OR Society',
            'job_url': url,
            'scrape_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'job_title': input_data['job_title'],
            'company_name': input_data['company_name'],
            'location': input_data['location'],
            'remote_option': '',
            'posting_date': input_data['posting_date'] or 'Not specified',
            'job_description_raw': input_data['description'],
            'job_description_cleaned': description_cleaned,
            'salary_info': '',
            'industry': 'Operations Research',
            'job_level': '',
            'gendered_words_found': '',
            'masculine_score': 0,
            'feminine_score': 0,
            'neutral_score': 0,
            'seniority_flag': '',
            'word_count': 0,
            'manual_review_flag': False,
            'ad_url': url
        }
        
        # Process salary info
        if input_data['salary_info']:
            job_data['salary_info'] = input_data['salary_info']
        else:
            job_data['salary_info'] = self.extract_salary_info(description_cleaned)
        
        # Determine remote option
        combined_text = f"{job_data['job_title']} {job_data['location']} {description_cleaned}"
        job_data['remote_option'] = self.detect_remote_option(combined_text)
        
        # Determine job level
        job_data['job_level'] = self.determine_job_level(job_data['job_title'], description_cleaned)
        job_data['seniority_flag'] = "Senior" in job_data['job_level']
        
        # Analyze gendered language
        masculine_score, feminine_score, neutral_score, gendered_words = self.analyze_gendered_language(description_cleaned)
        
        job_data['masculine_score'] = masculine_score
        job_data['feminine_score'] = feminine_score
        job_data['neutral_score'] = neutral_score
        job_data['gendered_words_found'] = '; '.join(gendered_words) if gendered_words else ''
        
        # Word count
        if description_cleaned:
            job_data['word_count'] = len(description_cleaned.split())
        
        # Manual review flag
        if (job_data['word_count'] < 20 or 
            not job_data['job_title'] or 
            not job_data['company_name']):
            job_data['manual_review_flag'] = True
        
        return job_data

    def save_to_excel(self, job_data, output_path, append_mode=False):
        """Save job data to Excel file"""
        try:
            # Ensure output directory exists
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # Create DataFrame
            df_new = pd.DataFrame([job_data])
            
            if append_mode and os.path.exists(output_path):
                # Read existing data and append
                try:
                    df_existing = pd.read_excel(output_path)
                    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
                except:
                    df_combined = df_new
            else:
                df_combined = df_new
            
            # Save to Excel
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                df_combined.to_excel(writer, sheet_name='Job_Data', index=False)
            
            print(f"\nData saved to: {output_path}")
            print(f"Total jobs in file: {len(df_combined)}")
            return True
            
        except Exception as e:
            print(f"Error saving to Excel: {e}")
            return False

def main():
    """Main function for manual job data entry"""
    
    # Configuration
    OUTPUT_PATH = r"C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Codes\or_society_jobs.xlsx"
    
    print("="*70)
    print("OR SOCIETY JOB DATA PROCESSOR - MANUAL ENTRY")
    print("="*70)
    print("This tool helps you manually enter job data and performs analysis.")
    print("You can process multiple jobs - the data will be appended to the Excel file.")
    print("="*70)
    
    processor = ORSocietyJobProcessor()
    
    while True:
        print("\n" + "="*40)
        print("NEW JOB ENTRY")
        print("="*40)
        
        # Get job URL
        url = input("Enter job URL: ").strip()
        if not url:
            print("No URL provided. Exiting...")
            break
        
        # Get job data
        input_data = processor.get_job_input(url)
        
        # Process the data
        job_data = processor.process_job_data(input_data, url)
        
        # Display results
        print("\n" + "="*60)
        print("PROCESSING RESULTS")
        print("="*60)
        print(f"Job Title: {job_data['job_title']}")
        print(f"Company: {job_data['company_name']}")
        print(f"Location: {job_data['location']}")
        print(f"Remote Option: {job_data['remote_option']}")
        print(f"Job Level: {job_data['job_level']}")
        print(f"Salary Info: {job_data['salary_info']}")
        print(f"Word Count: {job_data['word_count']}")
        print(f"Masculine Score: {job_data['masculine_score']}%")
        print(f"Feminine Score: {job_data['feminine_score']}%")
        print(f"Neutral Score: {job_data['neutral_score']}%")
        if job_data['gendered_words_found']:
            print(f"Gendered Words: {job_data['gendered_words_found']}")
        print(f"Manual Review Flag: {job_data['manual_review_flag']}")
        
        # Save to Excel
        append_mode = os.path.exists(OUTPUT_PATH)
        success = processor.save_to_excel(job_data, OUTPUT_PATH, append_mode)
        
        if success:
            print("\n✅ Job data saved successfully!")
        else:y
            https://theorsociety.careerwebsite.com/jobs/view/chief-information-officer/f14adde5-5289877234/?keywords=RESEARCH&pos_flt=0&location=&location_completion=&location_type=&location_text=&location_autocomplete=true&location_latlong=&radius=320&sort=position
        Chief Information Officer
            print("\n❌ Failed to save job data.")
        
        # Ask if user wants to add another job
        print("\n" + "-"*40)
        continue_choice = input("Do you want to add another job? (y/n): ").strip().lower()
        if continue_choice not in ['y', 'yes']:
            break
    
    print("\n" + "="*70)
    print("SESSION COMPLETED")
    print("="*70)
    print(f"All job data has been saved to: {OUTPUT_PATH}")
    print("Thank you for using the OR Society Job Data Processor!")

if __name__ == "__main__":
    main()

OR SOCIETY JOB DATA PROCESSOR - MANUAL ENTRY
This tool helps you manually enter job data and performs analysis.
You can process multiple jobs - the data will be appended to the Excel file.

NEW JOB ENTRY


Enter job URL:  https://theorsociety.careerwebsite.com/jobs/view/lead-data-scientist/f14adde5-5287058406/



JOB DATA ENTRY
URL: https://theorsociety.careerwebsite.com/jobs/view/lead-data-scientist/f14adde5-5287058406/
------------------------------------------------------------


Job Title:  Lead Data Scientist
Company Name:  UCAS
Location:  UK
Posting Date (optional):  
Salary Info (optional):  



Job Description:
(Paste the job description below. Press Enter twice when finished)
----------------------------------------


...


Job Description:
(Paste the job description below. Press Enter twice when finished)
----------------------------------------


  We’re looking for a skilled Data Analyst who can do more than just crunch numbers. You’ll be responsible for the end-to-end management of our data – from building and maintaining our data warehouse, to driving data quality, analysis, and reporting that directly supports business decisions.  You’ll work across the company, supporting data governance, compliance, and insight generation. There’s a strong focus on improving how data is collected, stored, and used – and making sure everyone across the business understands the value of it.  What You’ll Be Doing  Governance & Quality  Develop and maintain our Data Governance Framework  Own data quality monitoring, audits, and issue management  Ensure consistent, accurate, and reliable data across all systems  Data Integration & Warehousing  Build and manage a centralised data warehouse  Apply ETL processes to extract, transform and load data  Integrate data across platforms using APIs, Celigo, Make  Keep documentation updated in the IT Know


PROCESSING RESULTS
Job Title: Data Insights Analyst
Company: IntaPeople
Location: Cardiff, Cardiff County, United Kingdom
Remote Option: On-site
Job Level: Junior
Salary Info: Not specified
Word Count: 353
Masculine Score: 1.68%
Feminine Score: 1.4%
Neutral Score: 96.92%
Gendered Words: F:responsible; M:strong; M:manage; F:support; F:support; M:excel; F:team; M:strong; M:confident; M:excel; F:help
Manual Review Flag: False

Data saved to: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Codes\or_society_jobs.xlsx
Total jobs in file: 60

✅ Job data saved successfully!

----------------------------------------


Do you want to add another job? (y/n):  n



SESSION COMPLETED
All job data has been saved to: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Codes\or_society_jobs.xlsx
Thank you for using the OR Society Job Data Processor!


OR SOCIETY JOB DATA PROCESSOR - MANUAL ENTRY
This tool helps you manually enter job data and performs analysis.
You can process multiple jobs - the data will be appended to the Excel file.

NEW JOB ENTRY


Enter job URL:  https://theorsociety.careerwebsite.com/jobs/view/associate-director-ai-enablement-operations-lead/f14adde5-5263019386/



JOB DATA ENTRY
URL: https://theorsociety.careerwebsite.com/jobs/view/associate-director-ai-enablement-operations-lead/f14adde5-5263019386/
------------------------------------------------------------


Job Title:  Associate Director, AI Enablement & Operations Lead
Company Name:  S&P Global
Location:  London, United Kingdom
Posting Date (optional):  
Salary Info (optional):  



Job Description:
(Paste the job description below. Press Enter twice when finished)
----------------------------------------


...


JOB DATA ENTRY
URL: https://theorsociety.careerwebsite.com/jobs/view/commercial-operations-analyst/f14adde5-5269349305/?keywords=operation&pos_flt=0&location=&location_completion=&location_type=&location_text=&location_autocomplete=true&location_latlong=&radius=320&sort=position
------------------------------------------------------------


Job Title:  Commercial Operations Analyst
Company Name:  Dealstack Ltd
Location:  Putney, South West London, United Kingdom
Posting Date (optional):  
Salary Info (optional):  



Job Description:
(Paste the job description below. Press Enter twice when finished)
----------------------------------------


 The Opportunity Dealstack is a SaaS platform that powers private equity. Private equity deal making remains manual, chaotic and backward. Our goal is to end these outdated practices and to become the new collaboration standard in the industry. Dealstack is built by PE, for PE. Founded by experienced professionals from within the industry, our solution sits at the intersection of SaaS, fintech and legaltech.Dealstack has an ambitious, multi-product roadmap. The first product went live in 2023. It allows private equity professionals to easily onboard and maintain investment programmes. Sincethen, we have launched several new products and features creating a multi-use case operating system for private equity.Following strong commercial traction from leading industry players, we are looking for an ambitious individual, with proven quantitative skills and attention to detail and who thrives with data-based tasks, to support our commercial operations. The objective of the role is simple: to


PROCESSING RESULTS
Job Title: Commercial Operations Analyst
Company: Dealstack Ltd
Location: Putney, South West London, United Kingdom
Remote Option: On-site
Job Level: Senior
Salary Info: Not specified
Word Count: 885
Masculine Score: 1.23%
Feminine Score: 1.12%
Neutral Score: 97.65%
Gendered Words: M:ambitious; M:strong; M:ambitious; M:individual; F:support; M:objective; F:help; M:achieve; M:responsibility; F:team; F:share; F:team; F:team; F:support; F:understand; F:share; M:excel; M:strong; F:team; M:excel; M:control
Manual Review Flag: False

Data saved to: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Codes\or_society_jobs.xlsx
Total jobs in file: 20

✅ Job data saved successfully!

----------------------------------------
